## Задание 6.3

Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
- SVM
- наивный байесовский классификатор
- логистическая регрессия

In [1]:
import codecs
import re

In [2]:
def data_read(address,token):
    data = list()
    labels = list()
    
    with codecs.open(address, "r","utf_8_sig") as file:
        line = file.readline()
        tokens = re.findall(token,line) 
        labels.append(tokens[0])
        data.append(tokens[1::])
        while line:
            line = file.readline()
            tokens = re.findall(token,line) 
            if tokens != []:
                labels.append(tokens[0])
                data.append(tokens[1::])
    return data, labels

In [3]:
token = re.compile(r'\w+')

address_test = 'news_test.txt'
address_train = 'news_train.txt'

test_data, test_labels = data_read(address_test,token)
train_data, train_labels = data_read(address_train,token)

In [4]:
train_data = [' '.join(i) for i in train_data]
test_data = [' '.join(i) for i in test_data]

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(train_data)

TfidfVectorizer()

In [6]:
X_train = vectorizer.transform(train_data)
X_test = vectorizer.transform(test_data)

In [7]:
from sklearn import preprocessing  
enc = preprocessing.LabelEncoder() 

In [8]:
enc.fit(train_labels)
Y_train = enc.transform(train_labels)
Y_test = enc.transform(test_labels)

Логистическая регрессия

In [9]:
%%time
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, Y_train)
y_pred_lr = log_reg.predict(X_test)

Wall time: 54.6 s


c:\users\1\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
from sklearn import metrics
metrics.accuracy_score(Y_test, y_pred_lr)

0.8743333333333333

Наивный байесовский классификатор

In [11]:
%%time
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, Y_train)
y_pred_nb = nb.predict(X_test)

Wall time: 247 ms


In [12]:
metrics.accuracy_score(Y_test, y_pred_nb)

0.808